In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x00000273DB294240>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x00000273DE02F438>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x00000273DE02F4E0>)

In [0]:
input_width=28
input_height=28
input_channels=1
input_pixels=784

n_conv1=32
n_conv2=64
stride_conv1=1
stride_conv2=1
conv1_k=5
conv2_k=5
maxpool1_k=2
maxpool2_k=2

n_hidden=1024
n_output=10

input_size_to_hidden_layer=(input_width//(maxpool1_k*maxpool2_k)) * (input_height//(maxpool1_k*maxpool2_k)) * n_conv2

In [0]:
weights={
    "wc1":tf.Variable(tf.random_normal([conv1_k,conv1_k,input_channels,n_conv1])),
    "wc2":tf.Variable(tf.random_normal([conv2_k,conv2_k,n_conv1,n_conv2])),   #n_conv1 becomes the nuber of channels
    "wh1":tf.Variable(tf.random_normal([input_size_to_hidden_layer,n_hidden])), 
    "out":tf.Variable(tf.random_normal([n_hidden,n_output]))
  }

biases={
    'bc1':tf.Variable(tf.random_normal([n_conv1])),
    'bc2':tf.Variable(tf.random_normal([n_conv2])),
    'bh1':tf.Variable(tf.random_normal([n_hidden])),
    'bo':tf.Variable(tf.random_normal([n_output])),
}

In [0]:
def cnn(x,weights,biases,keep_probability):         #code for forward propagation
    x=tf.reshape(x,shape=[-1,input_width,input_height,input_channels])
    conv1=conv(x,weights['wc1'],biases['bc1'],stride_conv1)
    conv1_pool=maxpooling(conv1,maxpool1_k)
    conv2=conv(conv1_pool,weights['wc2'],biases['bc2'],stride_conv2)
    conv2_pool=maxpooling(conv2,maxpool2_k)
    
    hidden_input=tf.reshape(conv2_pool,shape=[-1,input_size_to_hidden_layer])
    hidden_output_before_activation=tf.add(tf.matmul(hidden_input,weights['wh1']),biases['bh1'])
    hidden_output_before_dropout=tf.nn.relu(hidden_output_before_activation)
    hidden_output=tf.nn.dropout(hidden_output_before_dropout,keep_probability)
    #we are using identity activation function for output layer
    output=tf.add(tf.matmul(hidden_output,weights['out']),biases['bo'])
    
    return output
    

In [0]:
def conv(x,weight,bias,strides=1):
    out=tf.nn.conv2d(x,weight,padding='SAME',strides=[1,strides,strides,1])
    out=tf.nn.bias_add(out,bias)
    out=tf.nn.relu(out)
    return out

def maxpooling(x,k=2):
    return tf.nn.max_pool(x,padding="SAME",ksize=[1,k,k,1], strides=[1,k,k,1])



In [0]:
x=tf.placeholder("float",[None,input_pixels])  #2nd parameter is of shape, we know that coloumns are 784
y=tf.placeholder(tf.int32,[None,n_output])        #int32 because our images have integars only 0-9
keep_probability=tf.placeholder("float")
pred=cnn(x,weights,biases,keep_probability)

In [0]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))

In [0]:
optimizer=tf.train.AdamOptimizer(learning_rate=0.01)
optimize=optimizer.minimize(cost)

In [0]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [12]:
#will take lot of time
batch_size=100
for i in range(25):
    num_batches=int(mnist.train.num_examples/batch_size)
    total_cost=0
    for j in range(num_batches):
        batch_x,batch_y = mnist.train.next_batch(batch_size)
        c,_=sess.run([cost,optimize],feed_dict={x:batch_x,y:batch_y,keep_probability:0.8})
        total_cost+=c
    print(total_cost)

1240055.9869561195
26852.93487853557
14730.625907872913
11049.147435409928
10300.727879573107
9550.534168239308
9839.744130756437
9111.054878336461
7067.859487171241
9222.807227719892
7699.520658216111
5619.764661180927
6676.402466848493
5006.996409666891
5468.277526299188
4856.440849905695
3817.6607206487006
3564.4198024998336
3024.798473127094
3193.7805132726776
3061.6307610346284
2473.5269766822457
2446.145207944237
2180.393599972911
1858.7178370924812


In [13]:
predictions=tf.argmax(pred,1)
true_labels=tf.argmax(y,1)
correct_predictions=tf.equal(predictions,true_labels)
predictions_eval,correct_pred=sess.run([predictions,correct_predictions],feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_probability:1.0})
correct_pred.sum()  

9857

In [ ]:
#all above can be done by using keras as well. Hint-Link in bookmarks(Medium)